In [8]:
from PIL import Image
import numpy as np                         
from skimage import measure
from shapely.geometry import Polygon, MultiPolygon
import json
import matplotlib.pyplot as plt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = "/content/drive/MyDrive/Machine learning/data.zip"
!unzip "/content/drive/MyDrive/Machine learning/data.zip"

Archive:  /content/drive/MyDrive/Machine learning/data.zip
   creating: data/
   creating: data/lesion/
   creating: data/lesion/P001/
  inflating: data/lesion/P001/10006.bmp  
  inflating: data/lesion/P001/10007.bmp  
  inflating: data/lesion/P001/10008.bmp  
  inflating: data/lesion/P001/10009.bmp  
  inflating: data/lesion/P001/10010.bmp  
  inflating: data/lesion/P001/10011.bmp  
  inflating: data/lesion/P001/10012.bmp  
  inflating: data/lesion/P001/10013.bmp  
  inflating: data/lesion/P001/10014.bmp  
  inflating: data/lesion/P001/10015.bmp  
  inflating: data/lesion/P001/10016.bmp  
  inflating: data/lesion/P001/10017.bmp  
  inflating: data/lesion/P001/10018.bmp  
  inflating: data/lesion/P001/10019.bmp  
  inflating: data/lesion/P001/10020.bmp  
  inflating: data/lesion/P001/10021.bmp  
   creating: data/lesion/P002/
  inflating: data/lesion/P002/10006.bmp  
  inflating: data/lesion/P002/10007.bmp  
  inflating: data/lesion/P002/10008.bmp  
  inflating: data/lesion/P002/10009.

In [3]:
def create_sub_masks(mask_image):
    width, height = mask_image.size

    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))[:3]

            # If the pixel is not white...(we have white background)
            if pixel != (255, 255, 255):
                # Check to see if we've created a sub-mask...
                pixel_str = str(pixel)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks

In [15]:
def create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    #     measure.find_contours
    #Uses the “marching squares” method to compute a the iso-valued contours of 
    #the input 2D array for a particular level value. 
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')
    bboxes = []
    segmentations = []
    polygons = []
    areas = []
    annotations = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        #shapely.geometry Polygon
        
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        segmentations.append(segmentation)
    # Combine the polygons to calculate the bounding box and area
    offset = 0
    steps = 0
    for i , polygon in enumerate(polygons):
        bbox = []
        area = []
        if not polygon.is_empty:
            bbox = polygon.bounds
            area = polygon.area
        annotation = {
        'segmentation': segmentations[i],
        'iscrowd': 0,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area}
        annotations.append(annotation)
        steps +=1
        annotation_id += 1
    return annotations , steps
        
        

In [22]:
errors=['./data/lesion/P009/10014.bmp',
 './data/lesion/P006/10011.bmp',
 './data/lesion/P058/10006.bmp',
 './data/lesion/P058/10016.bmp',
 './data/lesion/P032/10013.bmp',
 './data/lesion/P028/10007.bmp',
 './data/lesion/P013/10011.bmp',
 './data/lesion/P016/10012.bmp',
 './data/lesion/P016/10018.bmp']

In [63]:
import os
from tqdm import tqdm
data_path = "./data/"
out_path = "/content/drive/MyDrive/Machine learning/annotations.json"
paths=[]
for (root,dirs,files) in os.walk(data_path, topdown=True):
    for filename in files:
        if filename.endswith('.bmp'):
            path = os.path.join(root, filename)
            paths.append(path)
lesion_id = 1
is_crowd= 0
image_id = 1
annotation_id = 1
error_paths = []
first = True
offset = 0
for mask_image_path in tqdm(paths):
  if mask_image_path not in errors:
      with Image.open(mask_image_path) as mask_image:
          try:
            path_split = mask_image.filename.split('/')
            image_id = path_split[3]+"."+path_split[4].split('.')[0]
            print(image_id)
            sub_masks = create_sub_masks(mask_image)
            for color, sub_mask in sub_masks.items():
                category_id = lesion_id
                annotation, steps = create_sub_mask_annotation(sub_mask, image_id, category_id,annotation_id, is_crowd)
                annotation_id += steps
          except Exception as e:
            print(e,image_id)
            error_paths.append(mask_image_path)
          with open(out_path,'a') as outfile:
              x=json.dumps(annotation)
              x=x[offset:-1] #delete last character
              outfile.write(x)
              outfile.write(',')
              if first:
                first=False
                offset=1
else:
  with open(out_path,'rb+') as outfile:
    outfile.seek(-1, os.SEEK_END)
    outfile.truncate()
  with open(out_path,'a') as outfile:
     outfile.write(']')


                





  0%|          | 0/460 [00:00<?, ?it/s]

P050.10006






  0%|          | 1/460 [00:00<02:29,  3.07it/s]

P050.10012






  0%|          | 2/460 [00:00<02:25,  3.15it/s]

...





 99%|█████████▉| 457/460 [02:38<00:01,  2.94it/s]

P016.10009






100%|█████████▉| 458/460 [02:38<00:00,  2.89it/s]

P016.10014






100%|██████████| 460/460 [02:39<00:00,  2.89it/s]


ValueError: ignored

In [32]:
error_paths

[]

TypeError: ignored